In [1]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
# %% ------------------- PATHS ------------------- #
train_dir = r"D:\Military_Aircraft_CNN\Images\Train"
test_dir = r"D:\Military_Aircraft_CNN\Images\Test"   

In [3]:
# -----------------------------
# Image Data Generators (Augmentation)
# -----------------------------
train_df = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8,1.2],
    validation_split=0.2
)

train_generator = train_df.flow_from_directory(
    directory=train_dir,
    target_size=(224,224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_df.flow_from_directory(
    directory=train_dir,  # same directory, use validation split
    target_size=(224,224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 4536 images belonging to 81 classes.
Found 1134 images belonging to 81 classes.


In [4]:
# -----------------------------
# Load Pretrained ResNet101
# -----------------------------
num_classes = train_generator.num_classes
base_model = ResNet101(weights='imagenet', 
                       include_top=False, 
                       input_shape=(224,224,3))

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# -----------------------------
# Build Model
# -----------------------------
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet101 (Functional)          │ (None, 7, 7, 2048)     │    42,658,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 81)             │        41,553 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,757,009 (166.92 MB)

 Trainable params: 1,094,737 (4.18 MB)

 Non-trainable params: 42,662,272 (162.74 MB)

In [7]:
# Compile
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
# -----------------------------
# Callbacks (EarlyStopping + Save Best Model)
# -----------------------------
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "best_resnet101.keras",   # Save the best model
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [9]:
# -----------------------------
# Transfer Learning Training
# -----------------------------
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    callbacks=[early_stopping, checkpoint]
)

C:\Users\logan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.0162 - loss: 5.7878
Epoch 1: val_loss improved from None to 4.06064, saving model to best_resnet101.keras
142/142 ━━━━━━━━━━━━━━━━━━━━ 696s 5s/step - accuracy: 0.0251 - loss: 5.5099 - val_accuracy: 0.0961 - val_loss: 4.0606
Epoch 2/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.0660 - loss: 4.7120
Epoch 2: val_loss improved from 4.06064 to 3.59321, saving model to best_resnet101.keras
142/142 ━━━━━━━━━━━━━━━━━━━━ 654s 5s/step - accuracy: 0.0721 - loss: 4.6151 - val_accuracy: 0.2090 - val_loss: 3.5932
Epoch 3/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.1072 - loss: 4.2030
Epoch 3: val_loss improved from 3.59321 to 3.32172, saving model to best_resnet101.keras
142/142 ━━━━━━━━━━━━━━━━━━━━ 665s 5s/step - accuracy: 0.1199 - loss: 4.1506 - val_accuracy: 0.2469 - val_loss: 3.3217
Epoch 4/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.1596 - loss: 3.8149
Epoch 4: val_loss improved from 3.32172 to

In [10]:
# -----------------------------
# Fine-tuning: Unfreeze last 50 layers of ResNet101
# -----------------------------
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# -----------------------------
# Fine-tuning Training
# -----------------------------
history_finetune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    callbacks=[early_stopping, checkpoint]
)


Epoch 1/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4697 - loss: 1.9935
Epoch 1: val_loss improved from 2.04027 to 2.03770, saving model to best_resnet101.keras
142/142 ━━━━━━━━━━━━━━━━━━━━ 908s 6s/step - accuracy: 0.4773 - loss: 1.9894 - val_accuracy: 0.5053 - val_loss: 2.0377
Epoch 2/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4986 - loss: 1.8862
Epoch 2: val_loss improved from 2.03770 to 1.94959, saving model to best_resnet101.keras
142/142 ━━━━━━━━━━━━━━━━━━━━ 864s 6s/step - accuracy: 0.5009 - loss: 1.8614 - val_accuracy: 0.5132 - val_loss: 1.9496
Epoch 3/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.5030 - loss: 1.8530
Epoch 3: val_loss did not improve from 1.94959
142/142 ━━━━━━━━━━━━━━━━━━━━ 861s 6s/step - accuracy: 0.5148 - loss: 1.8248 - val_accuracy: 0.5071 - val_loss: 1.9551
Epoch 4/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.5399 - loss: 1.7551
Epoch 4: val_loss improved from 1.94959 to 1.92874, saving model to best_resnet10

In [11]:
# -----------------------------
# Final Evaluation
# -----------------------------
loss, acc = model.evaluate(validation_generator)
print(f"Validation Accuracy: {acc:.4f}, Validation Loss: {loss:.4f}")

36/36 ━━━━━━━━━━━━━━━━━━━━ 131s 4s/step - accuracy: 0.5661 - loss: 1.7547
Validation Accuracy: 0.5661, Validation Loss: 1.7547
